<a href="https://colab.research.google.com/github/jgermanob/COCID-PLN/blob/master/notebooks/Motor_de_b%C3%BAsqueda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6) Motor de búsqueda

In [ ]:
%%capture
!pip3 install datasets sentence-transformers
!pip3 install huggingface_hub

In [ ]:
from huggingface_hub import hf_hub_download
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch

#### 6.1) Descarga del conjunto de datos

In [ ]:
df = pd.read_csv(hf_hub_download("fabiochiu/medium-articles", repo_type="dataset", filename="medium_articles.csv"))
df = df[:1000].reset_index(drop=True)
df = df[['title', 'text', 'url']]
df.head()

#### 6.2) Descargar modelo pre-entrenado

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

#### 6.3) Vectorización de documentos

In [ ]:
corpus = df["text"].values
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
print(corpus_embeddings.shape)

torch.Size([1000, 384])


#### 6.4) Realización de consultas

In [ ]:
def relevant_documents(query_embedding, corpus_embeddings, df_articles, top_k=10):
  cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  position = 1
  for score, idx in zip(top_results[0], top_results[1]):
      row = df_articles.iloc[idx.item()]
      title = row["title"]
      print(f"{position} [score = {score}]: {title}")
      position += 1

Consulta utilizando únicamente palabras de contenido

In [ ]:
query = "data science nlp"
query_embedding = model.encode(query, convert_to_tensor=True)

In [ ]:
relevant_documents(query_embedding, corpus_embeddings, df)

Consulta utilizando palabras funcionales

In [ ]:
query = "how to learn NLP"

In [ ]:
query_embedding = model.encode(query, convert_to_tensor=True)

In [ ]:
relevant_documents(query_embedding, corpus_embeddings, df)